In [2]:
import torch
import tokenizers
from transformers import RobertaConfig, RobertaModel, AutoTokenizer, RobertaTokenizerFast
import pandas

C:\ProgramData\Anaconda3\envs\GenoPhen\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#file = r"/media/david/T7/NCBI/E.Coli.Shigella/PDG000000004.3840.amr.metadata.tsv"
file = r"D:\Datasets\NCBI\E.Coli_Shigella\PDG000000004.3840.amr.metadata.tsv"
raw_data = pandas.read_csv(file, sep='\t')

C:\Users\David\AppData\Local\Temp\ipykernel_9260\3468314195.py:3: DtypeWarning: Columns (1,5,6,30,55) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pandas.read_csv(file, sep='\t')


In [34]:
genotype_data = raw_data[~raw_data.AMR_genotypes_core.isnull()]
phenotype_data = raw_data[~raw_data.AST_phenotypes.isnull()]
pheno_geno_data = raw_data[(~raw_data.AST_phenotypes.isnull()) & (~raw_data.AMR_genotypes_core.isnull())]

In [29]:
def mask_three_geno_fn(x, min_len=3):
    all_items = x['AMR_genotypes_core'].split(',')
    all_items = [a for a in all_items if "=PARTIAL" not in a]
    all_items = [a for a in all_items if "=PARTIAL_END_OF_CONTIG" not in a]
    return len(all_items) >= min_len

In [30]:
def mask_two_geno_fn(x, min_len=2):
    all_items = x['AMR_genotypes_core'].split(',')
    all_items = [a for a in all_items if "=PARTIAL" not in a]
    all_items = [a for a in all_items if "=PARTIAL_END_OF_CONTIG" not in a]
    return len(all_items) >= min_len

In [49]:
def mask_one_pheno_fn(x, min_len=1):
    all_items = x['AST_phenotypes'].split(',')
    all_items = [a for a in all_items if "=ND" not in a]
    all_items = [a for a in all_items if "=" in a]
    return len(all_items) >= min_len

In [73]:
two_amr_one_pheno = pheno_geno_data[(pheno_geno_data.apply(mask_two_geno_fn, axis=1)) & (pheno_geno_data.apply(mask_one_pheno_fn, axis=1))]
print(two_amr_one_pheno.loc[2262]["AST_phenotypes"])
genotype_corpus = two_amr_one_pheno.AMR_genotypes_core.values.tolist()
genotype_corpus = [g.replace(',', ' ') for g in genotype_corpus]
phenotype_corpus = two_amr_one_pheno.AST_phenotypes.values.tolist()
phenotype_corpus = [g.replace(' ', '_') for g in phenotype_corpus] # Spaces already exists in some names, first replacing them with underscores
phenotype_corpus = [g.replace(',', ' ') for g in phenotype_corpus] #

dataset = []
for g, p in zip(genotype_corpus, phenotype_corpus):
    label_list = p.split(' ')
    pl = {}
    for l in label_list:
        nl = l.split('=')
        if nl[1] == "R":
            pl[nl[0]] = 1
        elif nl[1] == "S":
            pl[nl[0]] = 0
    pl["genes"] = g
    dataset.append(pl)
dataset[3]["genes"]

amikacin=ND,ampicillin=ND,ampicillin-sulbactam=ND,cefazolin=ND,cefotaxime=ND,cephalothin=ND,ciprofloxacin=ND,ertapenem=ND,gentamicin=ND,imipenem=ND,meropenem=R,tigecycline=ND


"aadA5 aph(3'')-Ib aph(6)-Id blaCTX-M-27 dfrA17 glpT_E448K=POINT gyrA_D87N=POINT gyrA_S83L=POINT mph(A) parC_E84V=POINT parC_S80I=POINT parE_I529L=POINT pmrB_E123D=POINT ptsI_V25I=POINT sul1 sul2 tet(A) uhpT_E350Q=POINT"

In [74]:
def get_genotype_corpus():
    for start_idx in range(0, len(genotype_corpus), 1000):
        samples = genotype_corpus[start_idx: start_idx + 1000]
        yield samples

In [75]:
training_genotype_corpus = get_genotype_corpus()
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base", add_prefix_space=True)
tokenizer.backend_tokenizer.pre_tokenizer = tokenizers.pre_tokenizers.CharDelimiterSplit(' ')
tokenizer.sep_token = None
tokenizer.bos_token = None
tokenizer.eos_token = None
tokenizer = tokenizer.train_new_from_iterator(training_genotype_corpus, 500000)
tokens = tokenizer.tokenize(dataset[3]["genes"])
tokens

['aadA5',
 "aph(3'')-Ib",
 'aph(6)-Id',
 'blaCTX-M-27',
 'dfrA17',
 'glpT_E448K=POINT',
 'gyrA_D87N=POINT',
 'gyrA_S83L=POINT',
 'mph(A)',
 'parC_E84V=POINT',
 'parC_S80I=POINT',
 'parE_I529L=POINT',
 'pmrB_E123D=POINT',
 'ptsI_V25I=POINT',
 'sul1',
 'sul2',
 'tet(A)',
 'uhpT_E350Q=POINT']

In [21]:
encoded_dict = tokenizer.encode_plus(
                example,       # Sentence to encode.
                add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                max_length = 10,           # Pad & truncate all sentences.
                padding = 'max_length',
                return_attention_mask = True,   # Construct attn. masks.
                return_tensors = 'pt',
                return_token_type_ids=False,# Return pytorch tensors.
                truncation=False,
                is_split_into_words=False)
input_ids = encoded_dict['input_ids']
print(input_ids.shape)
#tokenized = tokenizer.convert_ids_to_tokens([i.item() for i in input_ids.squeeze() if i > 1])
tokenized = tokenizer.convert_ids_to_tokens([i.item() for i in input_ids.squeeze()])
print(tokenized)

torch.Size([1, 10])
['<s>', 'glpT_E448K=POINT', 'marR_S3N=POINT', 'pmrB_E123D=POINT', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


In [57]:
raw_data[~raw_data.host_disease.isnull()]["host_disease"]

148       Hemorrhagic colitis, HUS
227                 not determined
228                       diarrhea
272                       diarrhea
273                       diarrhea
                    ...           
318690                Colonisation
318691     Urinary Tract Infection
318692                Colonisation
318693     Urinary Tract Infection
318694                Colonisation
Name: host_disease, Length: 61416, dtype: object

In [63]:
geno_pheno = genotype_data[~genotype_data.AST_phenotypes.isnull()]
geno_pheno[geno_pheno.apply(mask_two_geno_fn, axis=1)][geno_pheno.apply(mask_one_pheno_fn, axis=1)]

C:\Users\David\AppData\Local\Temp\ipykernel_12324\3765494842.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  geno_pheno[geno_pheno.apply(mask_two_geno_fn, axis=1)][geno_pheno.apply(mask_one_pheno_fn, axis=1)]


,#label,FDA_lab_id,HHS_region,IFSAC_category,LibraryLayout,PFGE_PrimaryEnzyme_pattern,PFGE_SecondaryEnzyme_pattern,Platform,Run,asm_acc,...,AMR_genotypes,AMR_genotypes_core,number_stress_genes,stress_genotypes,number_virulence_genes,virulence_genotypes,amrfinder_version,refgene_db_version,amrfinder_analysis_type,amrfinder_applied
1615,PDT000018993.1|SAMN02138670|GCA_000492235.1|Es...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCA_000492235.1,...,"aac(3)-VIa,aadA1,acrF,aph(3'')-Ib,aph(6)-Id,bl...","aac(3)-VIa,aadA1,aph(3'')-Ib,aph(6)-Id,blaHER-...",18,"pcoA,pcoB,pcoC,pcoD,pcoR,pcoS,qacEdelta1,silA,...",2,"espX1=PARTIAL,fdeC",3.11.14,2023-04-17.1,COMBINED,1
1625,PDT000019184.2|SAMN02138669|GCA_000492255.1|Es...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCA_000492255.1,...,"aadA5,acrF,blaEC,blaTEM-1,dfrA17,glpT_E448K=PO...","aadA5,blaTEM-1,dfrA17,glpT_E448K=POINT,gyrA_D8...",3,"emrE,qacEdelta1,ymgB",16,"afaC,fdeC,iha,iss,iucA,iucB,iucC,iucD,iutA,nfa...",3.11.14,2023-04-17.1,COMBINED,1
1738,PDT000020617.1|SAMN02138649|GCA_000522145.1|Es...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCA_000522145.1,...,"aadA5,acrF,aph(3'')-Ib,aph(6)-Id,blaEC,blaKPC-...","aadA5,aph(3'')-Ib,aph(6)-Id,blaKPC-3,dfrA17,gl...",7,"emrE,ncrA,ncrB,ncrC,ncrY,qacEdelta1,ymgB",14,"fdeC,iha,iss,iucA,iucB,iucC,iucD,iutA,papA=PAR...",3.11.14,2023-04-17.1,COMBINED,1
1739,PDT000020619.2|SAMN02138646|GCA_000522185.2|Es...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCA_000522185.2,...,"aac(3)-IIg,aac(6')-IIc,acrF,arr,blaEC,blaKPC-2...","aac(3)-IIg,aac(6')-IIc,arr,blaKPC-2,blaSHV-12,...",17,"arsA,arsD,arsR,emrE,merA,merC=HMM,merD,merE,me...",9,"eilA,espX1,fdeC,iss,lpfA,sinH=HMM,sslE=HMM,ybt...",3.10.40,2022-08-09.1,COMBINED,1
1740,PDT000020626.1|SAMN02138602|GCA_000522325.1|Es...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCA_000522325.1,...,"aadA5,acrF,aph(3'')-Ib,aph(6)-Id,blaCTX-M-15,b...","aadA5,aph(3'')-Ib,aph(6)-Id,blaCTX-M-15,cyaA_S...",3,"emrE,qacEdelta1,ymgB",13,"air=PARTIAL,eilA,espX1,fdeC,iucA,iucB,iucC,iuc...",3.11.14,2023-04-17.1,COMBINED,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315521,PDT001769948.1|SAMN28235850||Escherichia coli|...,NaN,NaN,NaN,PAIRED,NaN,NaN,ILLUMINA,SRR19200984,NaN,...,"aac(3)-IIe,aadA2,acrF,acrR_R45C=POINT,blaCTX-M...","aac(3)-IIe,aadA2,acrR_R45C=POINT,blaCTX-M-14,c...",3,"emrE,qacEdelta1,ymgB",7,"astA,eilA,espX1,fdeC,iss,ybtP,ybtQ",3.11.14,2023-04-17.1,NUCLEOTIDE,1
315522,PDT001769950.1|SAMN28235848||Escherichia coli|...,NaN,NaN,NaN,PAIRED,NaN,NaN,ILLUMINA,SRR19200986,NaN,...,"aac(3)-IId,aadA5,acrF,blaCTX-M-14,blaEC,dfrA17...","aac(3)-IId,aadA5,blaCTX-M-14,dfrA17,glpT_E448K...",3,"emrE,qacEdelta1,ymgB",15,"cnf1,fdeC,ireA,iss,papA,papC,papE,papF,papG-II...",3.11.14,2023-04-17.1,NUCLEOTIDE,1
315523,PDT001769952.1|SAMN28235846||Escherichia coli|...,NaN,NaN,NaN,PAIRED,NaN,NaN,ILLUMINA,SRR19200988,NaN,...,"aadA1,acrF,ant(2'')-Ia,blaCTX-M-14,blaEC,cmlA6...","aadA1,ant(2'')-Ia,blaCTX-M-14,cmlA6,cyaA_S352T...",2,"qacEdelta1,ymgB",16,"afaC,eilA,espX1,fdeC,iha,iss,iucA,iucB,iucC,iu...",3.11.14,2023-04-17.1,NUCLEOTIDE,1
315524,PDT001769956.1|SAMN28235834||Escherichia coli|...,NaN,NaN,NaN,PAIRED,NaN,NaN,ILLUMINA,SRR19200992,NaN,...,"aadA5,acrF,aph(3'')-Ib,aph(6)-Id,blaCTX-M-55,b...","aadA5,aph(3'')-Ib,aph(6)-Id,blaCTX-M-55,blaTEM...",6,"merC,merP,merR,merT,qacEdelta1,ymgB",9,"cvaC=PARTIAL,cvaC=PARTIAL_END_OF_CONTIG,espX1,...",3.11.14,2023-04-17.1,NUCLEOTIDE,1


In [65]:
geno_pheno[geno_pheno.apply(mask_two_geno_fn, axis=1)][geno_pheno.apply(mask_one_pheno_fn, axis=1)][~geno_pheno.host_disease.isnull()]["host_disease"]

C:\Users\David\AppData\Local\Temp\ipykernel_12324\2686540856.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  geno_pheno[geno_pheno.apply(mask_two_geno_fn, axis=1)][geno_pheno.apply(mask_one_pheno_fn, axis=1)][~geno_pheno.host_disease.isnull()]["host_disease"]
C:\Users\David\AppData\Local\Temp\ipykernel_12324\2686540856.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  geno_pheno[geno_pheno.apply(mask_two_geno_fn, axis=1)][geno_pheno.apply(mask_one_pheno_fn, axis=1)][~geno_pheno.host_disease.isnull()]["host_disease"]


2825                        Wound
2826                        Wound
22640                 skin lesion
26436     Urinary tract infection
26782                 Bacteriuria
                   ...           
278153                  Not known
278154                  Not known
286188              not collected
303750                shigellosis
314993    Urinary Tract Infection
Name: host_disease, Length: 2212, dtype: object